# Fizz Buzz

### Resources

This notebook is inspired/created by/from this [blog post](http://joelgrus.com/2016/05/23/fizz-buzz-in-tensorflow/) by Joel Grus.

### Note:

This isn't exactly a *serious* solution to a problem, but it's interesting+fun

In [1]:
# NOTE: this is a custom cell that contains the common imports I personally 
# use these may/may not be necessary for the following examples

# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os # handling file i/o
import sys
import math
import time # timing epochs
import random

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import colors # making colors consistent
from mpl_toolkits.axes_grid1 import make_axes_locatable # colorbar helper


# from imageio import imread # read image from disk
# + data augmentation
from scipy import ndimage
from scipy import misc


import pickle # manually saving best params
from sklearn.utils import shuffle # shuffling data batches
from tqdm import tqdm # display training progress bar

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} created".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
    
def make_standard_dirs(saver=True, best_params=True, tf_logs=True):
    # `saver/` will hold tf saver files
    maybe_create_dir("saver")
    # `best_params/` will hold a serialized version of the best params
    # I like to keep this as a backup in case I run into issues with
    # the saver files
    maybe_create_dir("best_params")
    # `tf_logs/` will hold the logs that will be visable in tensorboard
    maybe_create_dir("tf_logs")

    
# set tf log level to supress messages, unless an error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Important Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.6.0-dev20180105
No GPU


In [2]:
make_standard_dirs()

saver already exists
best_params already exists
tf_logs already exists


In [3]:
### Clean all logs
## WARNING! You likely don't want to do this (but if you do, this is a convenient call)
# !rm -r -f ./tf_logs/*

In [4]:
# these two functions (get_model_params and restore_model_params) are 
# ad[a|o]pted from; 
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
def get_model_params():
    global_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {global_vars.op.name: value for global_vars, value in 
            zip(global_vars, tf.get_default_session().run(global_vars))}

def restore_model_params(model_params, g, sess):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: g.get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    sess.run(assign_ops, feed_dict=feed_dict)

# these two functions are used to manually save the best
# model params to disk
def save_obj(obj, name):
    with open('best_params/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('best_params/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

## Create Data

In [5]:
BEST_PARAMS_PATH = "best_params"

In [162]:
def fizz_buzz_encode(i):
    # one-hot representation of representations
    # fizz, buzz, fizzbuzz, and NA
    if i % 15 == 0:
        return np.array([0,0,0,1]) # fizzbuzz
    elif i % 5 == 0:
        return np.array([0,1,0,0]) # fizz
    elif i % 3 == 0:
        return np.array([0,0,1,0]) # buzz
    else:
        return np.array([1,0,0,0]) # none/NA


In [198]:
def fizz_buzz_decode(enc):
    if enc[1] == 1:
        return "fizz"
    elif enc[2] == 1:
        return "buzz"
    elif enc[3] == 1:
        return "fizzbuzz"
    
def class_decode(enc):
    if enc[1] == 1:
        return 1
    elif enc[2] == 1:
        return 2
    elif enc[3] == 1:
        return 3
    else:
        return 0
    
def ind_class_decode(ind):
    if ind == 0:
        return 'None'
    elif ind == 1:
        return 'Fizz'
    elif ind == 2:
        return 'Buzz'
    elif ind == 3:
        return 'FizzBuzz'

In [238]:
def create_hyper_params():
    hypms = {}
    hypms['n_epochs'] = 10000
    hypms['batch_size'] = 128
    hypms['init_lr'] = 1e-4
    #hypms['fc_dropout'] = 0.5
    hypms['n_input'] = 12
    hypms['n_classes'] = 4
    hypms['n_hidden'] = 256
    return hypms
hypms = create_hyper_params()

In [9]:
NUM_DIGITS = 12
tr_X = np.arange(101,2**NUM_DIGITS)
tr_y = np.array([fizz_buzz_encode(i) for i in tr_X])

In [10]:
print("{}; {}->{}".format(tr_X[1], tr_y[1], fizz_buzz_decode(tr_y[1])))

102; [0 0 1 0]->buzz


## Let's convert to binary

In [11]:
def binary_encode(i, NUM_DIGITS):
    bs = ("{0:0" + str(NUM_DIGITS) + "b}").format(i) # convert to bin rep
    bsl = [int(i) for i in bs] # convert bin rep to list of int
    return np.array(bsl)
#print("{}->{}".format(10, binary_encode(10, 10)))

def binary_decode(enc):
    bin_str = "".join([str(i) for i in enc])
    return int(bin_str, 2)

In [12]:
tr_X = np.array([binary_encode(i, NUM_DIGITS) for i in tr_X])

we now have tr_X and tr_y where;

```tr_X = binary representation of int value
tr_y = encoded label; ie [0 0 1 0] = "buzz
```

## Build graph

In [13]:
tr_X, tr_y = shuffle(tr_X, tr_y, random_state=42)
X_tr = tr_X[500:]
y_tr = tr_y[500:]
X_val = tr_X[:500]
y_val = tr_y[:500]

In [14]:
f,b,fb,n = 0,0,0,0
for vec in y_tr:
    if vec[3] == 1:
        fb += 1
    elif vec[2] == 1:
        b += 1
    elif vec[1] == 1:
        f += 1
    elif vec[0] == 1:
        n += 1
class_weights = np.array([n, f, b, fb], dtype=np.float32) / len(y_tr)
print(class_weights)
# unique, counts = np.unique(y_tr, return_counts=True, axis=1)
# for i in unique:
#     print("{} -> {}".format(unique[i], counts[i]))

[ 0.53218883  0.13218884  0.26752505  0.06809728]


In [145]:
class_weights = np.array([1,100,200,300], dtype=np.float32)

In [150]:
from sklearn.utils import class_weight

In [171]:
jj = np.array([class_decode(i) for i in y_tr])

In [220]:
jack = class_weight.compute_class_weight('balanced', np.array([0,1,2,3]), jj)
#jack = class_weight.compute_class_weight(None, np.array([0,1,2,3]), jj)

In [222]:
print(jack)

[ 0.46975806  1.89123377  0.93449198  3.67121849]


In [223]:
X_test = np.array([binary_encode(i, hypms['n_input']) for i in range(100)])
y_test = np.array([fizz_buzz_encode(i)                for i in range(100)])

In [224]:
for i in range(16):
    print("{}: -> {}".format(binary_decode(X_test[i]), fizz_buzz_decode(y_test[i])))

0: -> fizzbuzz
1: -> None
2: -> None
3: -> buzz
4: -> None
5: -> fizz
6: -> buzz
7: -> None
8: -> None
9: -> buzz
10: -> fizz
11: -> None
12: -> buzz
13: -> None
14: -> None
15: -> fizzbuzz


In [226]:
he_init = tf.contrib.layers.variance_scaling_initializer()

In [239]:
def build_graph():
    global class_weights
    g = tf.Graph()
    with g.as_default():
        global hypms
        with tf.name_scope("model"):
            # inputs
            with tf.name_scope("inputs"):
                X = tf.placeholder(tf.float32, shape=[None, hypms['n_input']], name="data")
                y = tf.placeholder(tf.float32, shape=[None, hypms['n_classes']], name="labels") # Target

            with tf.name_scope("layers"):
                h_01 = tf.layers.dense(X, 256, name="dense_01", kernel_initializer=he_init, activation=tf.nn.elu)
                h_02 = tf.layers.dense(h_01, 128, name="dense_02", kernel_initializer=he_init, activation=tf.nn.elu)
                h_03 = tf.layers.dense(h_02, 64, name="dense_03", kernel_initializer=he_init, activation=tf.nn.elu)
                h_04 = tf.layers.dense(h_03, 32, name="dense_04", kernel_initializer=he_init, activation=tf.nn.elu)

            with tf.name_scope("output"):
                logits = tf.layers.dense(h_04, hypms['n_classes'], name="logits")
                cls_weight = tf.constant([[jack[0],
                                           jack[1],
                                           jack[2],
                                           jack[3]]])
                weight_per_label = tf.transpose(tf.matmul(y, tf.transpose(cls_weight)))
                Y_proba = tf.nn.softmax(logits, name="Y_proba")
                print(Y_proba)

            with tf.name_scope("train"):
                xent = tf.nn.softmax_cross_entropy_with_logits(logits=logits, 
                                                               labels=y)
                xentropy = weight_per_label*xent
                #print(tmp_xent)
                #data_tf = tf.convert_to_tensor(cls_wight_vec, np.float32)
                #print(data_tf)
                #xentropy = tmp_xent*class_weights
                batch_loss = tf.reduce_mean(xentropy)
                optimizer = tf.train.AdamOptimizer(learning_rate=hypms['init_lr'])
                training_op = optimizer.minimize(batch_loss)
        with tf.name_scope("aux"):
            with tf.name_scope("save_session"):
                init_global = tf.global_variables_initializer()
                init_local = tf.local_variables_initializer()
                saver = tf.train.Saver()
            with tf.name_scope("metrics"):
                with tf.name_scope("common"):
                    y_true_cls = tf.argmax(y,1)
                    y_pred_cls = tf.argmax(Y_proba,1)
                    correct_prediction = tf.equal(y_pred_cls, y_true_cls, name="correct_predictions")
                    batch_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                with tf.name_scope("train_metrics") as scope:
                    train_auc, train_auc_update = tf.metrics.auc(labels=y, predictions=Y_proba)
                    train_acc, train_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                    train_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                    train_met_reset_op = tf.variables_initializer(train_acc_vars, name="train_met_reset_op")
                with tf.name_scope("val_metrics") as scope:
                    val_auc, val_auc_update = tf.metrics.auc(labels=y, predictions=Y_proba)
                    val_acc, val_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                    val_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                    val_met_reset_op = tf.variables_initializer(val_acc_vars, name="val_met_reset_op")
                with tf.name_scope("test_metrics") as scope:
                    test_auc, test_auc_update = tf.metrics.auc(labels=y, predictions=Y_proba)
                    test_acc, test_acc_update = tf.metrics.accuracy(labels=y_true_cls, predictions=y_pred_cls)
                    test_acc_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                    test_acc_reset_op = tf.variables_initializer(test_acc_vars, name="test_met_reset_op")

                # =============================================== loss 
                with tf.name_scope("train_loss_eval") as scope:
                    train_mean_loss, train_mean_loss_update = tf.metrics.mean(batch_loss)
                    train_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                    train_loss_reset_op = tf.variables_initializer(train_loss_vars, name="train_loss_reset_op")
                with tf.name_scope("val_loss_eval") as scope:
                    val_mean_loss, val_mean_loss_update = tf.metrics.mean(batch_loss)
                    val_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                    val_loss_reset_op = tf.variables_initializer(val_loss_vars, name="val_loss_reset_op")
                with tf.name_scope("test_loss_eval")as scope:
                    test_mean_loss, test_mean_loss_update = tf.metrics.mean(batch_loss)
                    test_loss_vars = tf.contrib.framework.get_variables(scope, collection=tf.GraphKeys.LOCAL_VARIABLES)
                    test_loss_reset_op = tf.variables_initializer(test_loss_vars, name="test_loss_rest_op")
                
            with tf.name_scope("tensorboard_writer") as scope:
                epoch_train_loss_scalar = tf.summary.scalar('train_epoch_loss', train_mean_loss)
                epoch_train_acc_scalar = tf.summary.scalar('train_epoch_acc', train_acc)
                epoch_train_auc_scalar = tf.summary.scalar('train_epoch_auc', train_auc)
                epoch_train_write_op = tf.summary.merge([epoch_train_loss_scalar, epoch_train_acc_scalar, epoch_train_auc_scalar], name="epoch_train_write_op")

                # ===== epoch, validation
                epoch_validation_loss_scalar = tf.summary.scalar('validation_epoch_loss', val_mean_loss)
                epoch_validation_acc_scalar = tf.summary.scalar('validation_epoch_acc', val_acc)
                epoch_validation_auc_scalar = tf.summary.scalar('validation_epoch_auc', val_auc)
                epoch_validation_write_op = tf.summary.merge([epoch_validation_loss_scalar, epoch_validation_acc_scalar, epoch_validation_auc_scalar], name="epoch_validation_write_op")
        
        # --- create collections
        for node in (saver, init_global, init_local):
            g.add_to_collection("save_init", node)
        for node in (X, y, training_op):
            g.add_to_collection("main_ops", node)
        for node in (Y_proba, y_true_cls, y_pred_cls, correct_prediction):
            g.add_to_collection("preds", node)
        for node in (train_auc, train_auc_update, train_acc, train_acc_update, train_met_reset_op):
            g.add_to_collection("train_metrics", node)
        for node in (val_auc, val_auc_update, val_acc, val_acc_update, val_met_reset_op):
            g.add_to_collection("val_metrics", node)
        for node in (test_auc, test_auc_update, test_acc, test_acc_update, test_acc_reset_op):
            g.add_to_collection("test_metrics", node)
        for node in (train_mean_loss, train_mean_loss_update, train_loss_reset_op):
            g.add_to_collection("train_loss", node)
        for node in (val_mean_loss, val_mean_loss_update, val_loss_reset_op):
            g.add_to_collection("val_loss", node)
        for node in (test_mean_loss, test_mean_loss_update, test_loss_reset_op):
            g.add_to_collection("test_loss", node)
        g.add_to_collection("logits", logits)
        for node in (epoch_train_write_op, epoch_validation_write_op):
                g.add_to_collection("tensorboard", node)
                

    return g

In [240]:
def train_graph(g):
    global hypms
    global BEST_PARAMS_PATH
    global X_tr, y_tr
    saver, init_global, init_local = g.get_collection("save_init")
    X, y, training_op = g.get_collection("main_ops")
    preds, y_true_cls, y_pred_cls, _ = g.get_collection("preds")
    train_auc, train_auc_update, train_acc, train_acc_update, train_met_reset_op = g.get_collection("train_metrics")
    val_auc, val_auc_update, val_acc, val_acc_update, val_met_reset_op = g.get_collection("val_metrics")
    train_mean_loss, train_mean_loss_update, train_loss_reset_op = g.get_collection("train_loss")
    val_mean_loss, val_mean_loss_update, val_loss_reset_op = g.get_collection("val_loss")
    epoch_train_write_op, epoch_validation_write_op = g.get_collection("tensorboard")
    train_writer = tf.summary.FileWriter(os.path.join("tf_logs","train"))
    val_writer = tf.summary.FileWriter(os.path.join("tf_logs","validation"))
    best_val_loss = np.inf
    
    with tf.Session(graph=g) as sess:
        sess.run([init_global, init_local])
        #train_writer.add_summary
        for e in tqdm(range(1,hypms['n_epochs']+1)):
            sess.run([val_met_reset_op,val_loss_reset_op,train_met_reset_op,train_loss_reset_op])
            
            X_tr, y_tr = shuffle(X_tr, y_tr, random_state=42)
            for iteration in range(len(X_tr) // hypms['batch_size']):
                # last batch is ignored (but the above shuffle will integrate it
                # in the next round)
                low_i = iteration*hypms['batch_size']
                high_i = (iteration+1)*hypms['batch_size']
                X_batch = X_tr[low_i:high_i]     
                y_batch = y_tr[low_i:high_i]
                
                sess.run([training_op, train_auc_update, train_acc_update, train_mean_loss_update], 
                         feed_dict={X:X_batch, y:y_batch})

            # write average for epoch + graph information
            # run options is not working as expected -- my directory link must be broken
            #run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary = sess.run(epoch_train_write_op, run_metadata=run_metadata)
            train_writer.add_run_metadata(run_metadata, str(e)) # not sure on this..
            train_writer.add_graph(g)
            train_writer.add_summary(summary, e)
            train_writer.flush()

            # run validation
            for iteration in range(len(X_val) // hypms['batch_size']):
                low_i = iteration*hypms['batch_size']
                high_i = (iteration+1)*hypms['batch_size']
                Xvb = X_val[low_i:high_i]     
                yvb = y_val[low_i:high_i]
                sess.run([val_auc_update, val_acc_update, val_mean_loss_update], 
                                  feed_dict={X:Xvb, y:yvb})

            Xvb = X_val[high_i:]
            yvb = y_val[high_i:]
            if len(Xvb) > 0:
                sess.run([val_auc_update, val_acc_update, val_mean_loss_update], 
                         feed_dict={X: Xvb, y: yvb})

                
            # check for (and save) best validation params here
            cur_loss, cur_acc = sess.run([val_mean_loss, val_acc])
            if cur_loss < best_val_loss:
                best_val_loss = cur_loss
                best_params = get_model_params()
                save_obj(best_params, BEST_PARAMS_PATH)
                print("best params saved: val acc: {:.3f}% val loss: {:.4f}".format(cur_acc*100, cur_loss))
            
            summary = sess.run(epoch_validation_write_op) 
            val_writer.add_summary(summary, e)
            val_writer.flush()

        # close writers
        train_writer.close()
        val_writer.close()
    return sess

In [241]:
reset_graph()
hypms = create_hyper_params()
g = build_graph()
sess = train_graph(g)

Tensor("model/output/Y_proba:0", shape=(?, 4), dtype=float32)


  0%|          | 1/10000 [00:00<1:16:30,  2.18it/s]

best params saved: val acc: 23.800% val loss: 1.5548


  0%|          | 2/10000 [00:00<1:07:47,  2.46it/s]

best params saved: val acc: 26.800% val loss: 1.4464


...

best params saved: val acc: 99.400% val loss: 0.0284


  7%|▋         | 666/10000 [04:04<57:10,  2.72it/s]

best params saved: val acc: 99.400% val loss: 0.0274


  7%|▋         | 733/10000 [04:26<56:10,  2.75it/s]

KeyboardInterrupt: 

In [242]:
reset_graph()
hypms = create_hyper_params()
g_eval = build_graph()
best_params = load_obj(BEST_PARAMS_PATH)
pred_dict = {}
with tf.Session(graph=g_eval) as sess:
    saver, init_global, init_local = g_eval.get_collection("save_init")
    X, y, training_op = g_eval.get_collection("main_ops")
    Y_proba, y_true_cls, y_pred_cls, _ = g_eval.get_collection("preds")
    test_auc, test_auc_update, test_acc, test_acc_update, test_acc_reset_op = g_eval.get_collection("test_metrics")
    test_mean_loss, test_mean_loss_update, test_loss_reset_op = g_eval.get_collection("test_loss")
    
    restore_model_params(model_params=best_params, g=g_eval, sess=sess)
    sess.run([test_acc_reset_op, test_loss_reset_op])

    # run test
    low_i = 0
    high_i = 0
    for iteration in range(len(X_test) // hypms['batch_size']):
        low_i = iteration*hypms['batch_size']
        high_i = (iteration+1)*hypms['batch_size']
        Xtb = X_test[low_i:high_i]     
        ytb = y_test[low_i:high_i]
        sess.run([test_auc_update, test_acc_update, test_mean_loss_update], 
                     feed_dict={X:Xtb, y:ytb})
        cur_preds = sess.run(Y_proba, feed_dict={X: Xtb, y: ytb})
        print(cur_preds)
        for i, v in enumerate(Xtb):
            pred_dict[binary_decode(v)] = fizz_buzz_decode(cur_preds[i])
    
    Xtb = X_test[high_i:]
    ytb = y_test[high_i:]
    if len(Xtb) > 0:
        sess.run([test_auc_update, test_acc_update, test_mean_loss_update], 
                 feed_dict={X: Xtb, y: ytb})
        cur_preds = sess.run(Y_proba, feed_dict={X: Xtb, y: ytb})
        print(cur_preds)
        for i, v in enumerate(Xtb):
            pred_dict[binary_decode(v)] = ind_class_decode(np.argmax(cur_preds[i]))
        
    # print
    final_test_acc, final_test_loss, final_test_auc = sess.run([test_acc, test_mean_loss, test_auc])
    print("test auc: {:.3f}% acc: {:.3f}% loss: {:.5f}".format(final_test_auc*100, 
                                                              final_test_acc*100,
                                                              final_test_loss))

Tensor("model/output/Y_proba:0", shape=(?, 4), dtype=float32)
[[  8.86257112e-01   5.05328132e-03   7.79468985e-03   1.00894980e-01]
 [  2.91699320e-01   6.77601218e-01   3.06994542e-02   9.41671914e-08]
 [  2.10215494e-01   2.08207339e-01   5.81576645e-01   5.53621078e-07]
 [  8.66933726e-04   3.84739451e-06   9.99129236e-01   1.54160545e-10]
 [  4.14531864e-02   9.40195322e-01   1.83513947e-02   1.18437733e-07]
 [  1.30701255e-05   9.99234200e-01   7.52765685e-04   2.01820743e-12]
 [  7.98468664e-03   9.58644506e-03   9.82428849e-01   4.88147087e-08]
 [  9.99857306e-01   2.89295494e-05   1.07260930e-04   6.48994273e-06]
 [  9.97536302e-01   2.42313463e-03   3.57183890e-05   4.87980924e-06]
 [  1.93903354e-04   2.36115558e-03   9.97444868e-01   1.32860570e-10]
 [  2.85350216e-05   9.99517560e-01   4.53910558e-04   4.19703694e-09]
 [  9.99856710e-01   1.33093868e-04   1.00954394e-05   1.17483818e-08]
 [  2.86269846e-04   1.74742854e-05   9.99696255e-01   1.69794144e-11]
 [  9.99874353e

In [243]:
for k in pred_dict:
    print(k,pred_dict[k])

0 None
1 Fizz
2 Buzz
3 Buzz
4 Fizz
5 Fizz
6 Buzz
7 None
8 None
9 Buzz
10 Fizz
11 None
12 Buzz
13 None
14 None
15 FizzBuzz
16 Buzz
17 Buzz
18 Buzz
19 None
20 Fizz
21 Buzz
22 None
23 None
24 Buzz
25 Fizz
26 None
27 Buzz
28 None
29 None
30 FizzBuzz
31 None
32 Buzz
33 Buzz
34 Buzz
35 Fizz
36 Buzz
37 None
38 None
39 Buzz
40 Fizz
41 None
42 Buzz
43 None
44 None
45 FizzBuzz
46 None
47 None
48 Buzz
49 None
50 Fizz
51 Buzz
52 None
53 None
54 Buzz
55 Fizz
56 None
57 Buzz
58 None
59 None
60 FizzBuzz
61 None
62 None
63 Buzz
64 Fizz
65 Fizz
66 Buzz
67 None
68 Buzz
69 Buzz
70 Fizz
71 None
72 Buzz
73 None
74 None
75 FizzBuzz
76 None
77 None
78 Buzz
79 None
80 Fizz
81 Buzz
82 None
83 None
84 Buzz
85 Fizz
86 None
87 Buzz
88 None
89 None
90 FizzBuzz
91 None
92 None
93 Buzz
94 None
95 Fizz
96 Buzz
97 None
98 None
99 Buzz
